In [ ]:
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
from model.convolutional_layer import read_convolutional_data, create_power_pipeline, create_runtime_pipeline

path = Path("data/mobilenet_v2/convolutional.csv")

input_features, power, runtime = read_convolutional_data(path)
input_features_train, input_features_test, power_train, power_test, runtime_train, runtime_test = train_test_split(input_features, power, runtime)

power_pipeline = create_power_pipeline()
runtime_pipeline = create_runtime_pipeline()

print(power_pipeline)
power_pipeline.fit(input_features_train.values, power_train.values)

power_pred = power_pipeline.predict(input_features_test.values)

display("Power Pipeline")
display(power_pipeline)
display("MSE", mean_squared_error(power_test, power_pred))
display("R2", r2_score(power_test, power_pred))

runtime_pipeline.fit(input_features_train.values, runtime_train.values)

runtime_pred = runtime_pipeline.predict(input_features_test.values)

display("Runtime Pipeline")
display(runtime_pipeline)
display("MSE", mean_squared_error(runtime_test, runtime_pred))
display("R2", r2_score(runtime_test, runtime_pred))


In [ ]:
from model.dense_layer import read_dense_data, create_power_pipeline, create_runtime_pipeline

path = Path("data/dummy_dense.csv")

input_features, power, runtime = read_dense_data(path)
input_features_train, input_features_test, power_train, power_test, runtime_train, runtime_test = train_test_split(input_features, power, runtime)

power_pipeline = create_power_pipeline()
runtime_pipeline = create_runtime_pipeline()

power_pipeline.fit(input_features_train.values, power_train.values)

power_pred = power_pipeline.predict(input_features_test.values)

display("Power Pipeline")
display(power_pipeline)
display("MSE", mean_squared_error(power_test, power_pred))
display("R2", r2_score(power_test, power_pred))

runtime_pipeline.fit(input_features_train.values, runtime_train.values)

runtime_pred = runtime_pipeline.predict(input_features_test.values)

display("Runtime Pipeline")
display(runtime_pipeline)
display("MSE", mean_squared_error(runtime_test, runtime_pred))
display("R2", r2_score(runtime_test, runtime_pred))


In [ ]:
from model.pooling_layer import read_pooling_data, create_power_pipeline, create_runtime_pipeline

path = Path("data/mobilenet_v2/pooling.csv")

input_features, power, runtime = read_pooling_data(path)
input_features_train, input_features_test, power_train, power_test, runtime_train, runtime_test = train_test_split(input_features, power, runtime)

power_pipeline = create_power_pipeline()
runtime_pipeline = create_runtime_pipeline()

power_pipeline.fit(input_features_train.values, power_train.values)

power_pred = power_pipeline.predict(input_features_test.values)

display("Power Pipeline")
display(power_pipeline)
display("MSE", mean_squared_error(power_test, power_pred))
display("R2", r2_score(power_test, power_pred))

runtime_pipeline.fit(input_features_train.values, runtime_train.values)

runtime_pred = runtime_pipeline.predict(input_features_test.values)

display("Runtime Pipeline")
display(runtime_pipeline)
display("MSE", mean_squared_error(runtime_test, runtime_pred))
display("R2", r2_score(runtime_test, runtime_pred))
